In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Enron-email

In [ ]:
email_org <- read.csv("../input/enron-email-dataset/emails.csv")


In [ ]:
head(email_org)
nrow(email_org)
head(email_structured)
nrow(email_structured)

In [ ]:
library(dplyr)
##将message列分为两列,并重命名，按X-From:来分割数据
# Split at X-From since there is no spacing at X-From.
email <- separate(email_org, col = "message", into = c("Message-ID", "X-From"), sep = "X-From:")

email <- separate(email, col = "Message-ID", into = c("Message-ID", "Date"), sep = "Date:")
email <- separate(email, col = "Date", into = c("Date", "From"), sep = "From:")
email <- separate(email, col = "From", into = c("From", "To"), sep = "To:")
email <- separate(email, col = "To", into = c("To", "Subject"), sep = "Subject:")
email <- separate(email, col = "Subject", into = c("Subject", "Mime-Version"), sep = "Mime-Version:")
email <- separate(email, col = "Mime-Version", into = c("Mime-Version", "Content-Type"), sep = "Content-Type:")
email <- separate(email, col = "Content-Type", into = c("Content-Type", "Content-Transfer-Encoding"), sep = "Content-Transfer-Encoding:")
# Separate the rest of the columns from X-From and onwards
email <- separate(email, col = "X-From", into = c("X-From", "X-To"), sep = "X-To:")
email <- separate(email, col = "X-To", into = c("X-To", "X-cc"), sep = "X-cc:")
email <- separate(email, col = "X-cc", into = c("X-cc", "X-bcc"), sep = "X-bcc:")
email <- separate(email, col = "X-bcc", into = c("X-bcc", "X-Folder"), sep = "X-Folder:")
email <- separate(email, col = "X-Folder", into = c("X-Folder", "X-Origin"), sep = "X-Origin:")
email <- separate(email, col = "X-Origin", into = c("X-Origin", "X-Filename"), sep = "X-FileName:")

# Difficult to split the message, use the file extension to split
email <- separate(email, col = "X-Filename", c("X-Filename", "message"), sep = ".(pst|nsf|PST|NSF)")

In [ ]:
head(email)

In [ ]:
write.csv(email, "/kaggle/working/Cleaned_Email.csv")
colnames(email)

In [ ]:
email_main = email[c('From', 'To', 'Subject', 'message')]
head(email_main)

In [ ]:
# Merge the subject into the text body
email_main$full_email <- paste(email_main$Subject, email_main$message, sep = ' ')
# Drop the separate columns
email_main <- subset(email_main, select = -c(Subject, message))

head(email_main)
dim(email_main)

In [ ]:
email_poi_from <- left_join(email_important, poi, by = c('From' = 'email_address'))
email_poi <- left_join(email_poi_from, poi, by = c('To' = 'email_address'))
write.csv(email_poi, "/kaggle/working/Email_poi_dataset.csv")
dim(email_poi)
head(email_poi)

# part2 Enron-Person-Of-Interest

The Enron scandal was a financial scandal that eventually led to the bankruptcy of the Enron Corporation, an American energy company based in Houston, Texas, and the de facto dissolution of Arthur Andersen, which was one of the five largest audit and accountancy partnerships in the world. In addition to being the largest bankruptcy reorganization in American history at that time, Enron was cited as the biggest audit failure.

Enron was formed in 1985 by Kenneth Lay after merging Houston Natural Gas and InterNorth. Several years later, when Jeffrey Skilling was hired, he developed a staff of executives that – by the use of accounting loopholes, special purpose entities, and poor financial reporting – were able to hide billions of dollars in debt from failed deals and projects. Chief Financial Officer Andrew Fastow and other executives not only misled Enron's Board of Directors and Audit Committee on high-risk accounting practices, but also pressured Arthur Andersen to ignore the issues.

Enron shareholders filed a 40 billion dollar lawsuit after the company's stock price, which achieved a high of US90.75 per share in mid-2000, plummeted to less than 1 dollar by the end of November 2001. The U.S. Securities and Exchange Commission (SEC) began an investigation, and rival Houston competitor Dynegy offered to purchase the company at a very low price. The deal failed, and on December 2, 2001, Enron filed for bankruptcy under Chapter 11 of the United States Bankruptcy Code. Enron's 63.4 billion dollars in assets made it the largest corporate bankruptcy in U.S. history until WorldCom's bankruptcy the next year.

Many executives at Enron were indicted for a variety of charges and some were later sentenced to prison. Enron's auditor, Arthur Andersen, was found guilty in a United States District Court of illegally destroying documents relevant to the SEC investigation which voided its license to audit public companies, effectively closing the business. By the time the ruling was overturned at the U.S. Supreme Court, the company had lost the majority of its customers and had ceased operating. Enron employees and shareholders received limited returns in lawsuits, despite losing billions in pensions and stock prices. As a consequence of the scandal, new regulations and legislation were enacted to expand the accuracy of financial reporting for public companies. One piece of legislation, the Sarbanes–Oxley Act, increased penalties for destroying, altering, or fabricating records in federal investigations or for attempting to defraud shareholders. The act also increased the accountability of auditing firms to remain unbiased and independent of their clients.

**Project Goal**

The goal of this project is to use the Enron dataset to train our machine learning algorithm to detect the possiblity of fraud (identify person's of interest.) Since we know our persons of interest (POIs) in our dataset, we will be able to use supervised learning algorithms in constructing our POI identifier. This will be done by picking the features within our dataset that separate our POIs from our non-POIs best.

We will start out our analysis by answering some questions about our data. Then, we will explore our features further by visualizing any correlations/outliers. Next, we will transform/scale our features and select those that will be most useful in our POI identifier, engineering new features and adding them to the dataset if provided to be useful for our analysis. We will identify at least two algorithms that may be best suited for our particular set of data and test them, tuning our parameters until optimal performance is reached. In our final analysis, the algorithm we have fit will be validated using our training/testing data. Using performance metrics to evaluate our results, any problems will be addressed and motifications made. In our final thoughts, the performance of our final algorithm will be discussed. 

# Gain dataset

In [40]:
enron = read.csv("../input/Enron-poi/enron.csv")

In [2]:
head(enron)
print("The number of employees:")
nrow(enron)
enron$name
print("The size of poi tables:")
dim(enron)
print("The name of each col:")
colnames(enron)
print("The number of non-poi vs poi:")
table(enron$poi)
print("The info of $poi:")
summary(enron$poi)

# poi - EDA

In [3]:
summary(enron)

In [4]:
library(DataExplorer)
plot_histogram(enron)
plot_density(enron)

**added feature, fraction of e-mails to and from poi**

In [41]:
enron$fraction_to_poi = enron$from_this_person_to_poi/enron$from_messages
enron$fraction_from_poi = enron$from_poi_to_this_person/enron$to_messages
#enron <- enron[,c(-3,-14,-16,-22)]

In [15]:
colnames(enron)
dim(enron)

**added feature, scaled total compensation**

In [42]:
enron$total_millions = enron$total_payments/enron$total_stock_value
#enron <- enron[,c(-4,-10)]

In [17]:
colnames(enron)
dim(enron)

**added feature, total bonus and exercised_stock_options**

In [47]:
enron$total_be = enron$bonus+enron$exercised_stock_options

In [ ]:
colnames(enron)
dim(enron)

In [25]:
head(enron)

**non-poi VS poi**

In [48]:
non_poi = subset(enron, enron$poi=="False")
poi = subset(enron, enron$poi=="True")
dim(non_poi)
print("summary of non_poi")
summary(non_poi)
head(non_poi)
dim(poi)
print("summary of poi")
summary(poi)
head(poi)

有18个poi，127个non_poi

**查看poi，non_poi往来邮件信息，对比**

In [49]:
non_poi_email_behavior = non_poi[c('shared_receipt_with_poi','to_messages','from_messages','fraction_from_poi','fraction_to_poi')]
dim(non_poi_email_behavior)
summary(non_poi_email_behavior)
poi_email_behavior = poi[c('shared_receipt_with_poi','to_messages','from_messages','fraction_from_poi','fraction_to_poi')]
dim(poi_email_behavior)
summary(poi_email_behavior)

与poi往来邮件数量最多的人.
我觉得看到一个人100%的邮件都发给了相关人员很有趣。下面，我打印出了这个人的一些特征。经过一番调查，我发现吉恩·汉弗莱是安然公司的第一批员工之一。所以，他所有的电子邮件都是发给从一开始就和公司在一起的相关人员的，这就说得通了。他们是他唯一的合作伙伴。

In [50]:
Max_fraction_to_poi = subset(enron, enron$fraction_to_poi=="1")
head(Max_fraction_to_poi)

**查看poi，non_poi经济信息，对比**

In [54]:
non_poi_money = non_poi[c('salary','bonus','exercised_stock_options','total_stock_value','total_payments','total_be','total_millions')]
dim(non_poi_money)
summary(non_poi_money)
head(non_poi_money)
poi_money = poi[c('salary','bonus','exercised_stock_options','total_stock_value','total_payments','total_be','total_millions')]
dim(poi_money)
summary(poi_money)
head(poi_money)

# Explore Features

In [ ]:
library("ggplot2")
summary(enron$salary)
ggplot(enron, aes(x = salary, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "salary", 
       x = "salary", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$bonus)
ggplot(enron, aes(x = bonus, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "bonus", 
       x = "bonus", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_payments)
ggplot(enron, aes(x = total_payments, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_payments", 
       x = "total_payments", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$exercised_stock_options)
ggplot(enron, aes(x = exercised_stock_options, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "exercised_stock_options", 
       x = "exercised_stock_options", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_be)
ggplot(enron, aes(x = total_be, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_be", 
       x = "total_be", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_stock_value)
ggplot(enron, aes(x = total_stock_value, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_stock_value", 
       x = "total_stock_value", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_millions)
ggplot(enron, aes(x = total_millions, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_millions", 
       x = "total_millions", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_millions)
ggplot(enron, aes(x = total_millions, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_millions", 
       x = "total_millions", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$total_millions)
ggplot(enron, aes(x = total_millions, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_millions", 
       x = "total_millions", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$shared_receipt_with_poi)
ggplot(enron, aes(x = shared_receipt_with_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "shared_receipt_with_poi", 
       x = "shared_receipt_with_poi", 
       y = "Density", 
       col = "poi") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

In [ ]:
library("ggplot2")
summary(enron$to_messages)
ggplot(enron, aes(x = to_messages, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "to_messages", 
       x = "to_messages", 
       y = "Density", 
       col = "Class") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$from_messages)
ggplot(enron, aes(x = from_messages, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "from_messages", 
       x = "from_messages", 
       y = "Density", 
       col = "Class") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$fraction_to_poi)
ggplot(enron, aes(x = fraction_to_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "fraction_to_poi", 
       x = "fraction_to_poi", 
       y = "Density", 
       col = "Class") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()

summary(enron$fraction_from_poi)
ggplot(enron, aes(x = fraction_from_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "fraction_from_poi", 
       x = "fraction_from_poi", 
       y = "Density", 
       col = "Class") +
  scale_fill_discrete(labels = c("True", "False"))+
  theme_light()